D팀 2주차

2주차 진행내용
1. 보다 명확한 조사 방향 설정을 위해, 페르소나 수정 및 구체화.
2. 상품목록 및 리뷰 데이터 크롤링 코드 보완 및 완성
3. 데이터 분석방향 설정



---
**1. 페르소나 수정 및 구체화.**


- **시장 조사**
    - **목표**: 시장에서 어떤 유형의 파자마가 인기 있는지 분석.
    - **방법**: 플랫폼 데이터를 크롤링하여 트렌드와 인기 제품 파악.
    
                    - 패션플랫폼 5개 TOP 30 조사 및 상위 브랜드 집중 분석
    
    - **결론**: "00 유형(소재/디자인/색상/가격) 파자마가 시장에서 인기
- **구매 의도 분석**
    - **목표**: 고객들이 특정 파자마를 구매한 의도와 목적을 파악.
    - **방법**: 리뷰 데이터를 통해 구매 동기 및 사용 목적 분석.
    - **결론**: 고객들은 대체적으로 "00”한 목적으로 파자마를 구매
- **시장 규모 평가**
    - **목표**: 해당 파자마 유형의 시장 규모 추정.
    - **방법**: 기타 뉴스, 리포트 분석 및 특정 파자마 플랫폼 판매량 취합
    - **결론**: "00한 파자마"의 시장 규모는 대략적으로 ~~로 추산.

- **브랜드 컨셉 및 판매 목표 설정**
    - **목표**: 위의 분석 결과를 기반으로 브랜드 컨셉을 수립하고 판매 전략을 제시.
    - **전략**:
        - "00한 컨셉" 파자마와 런칭 캠페인(구매의도 고려) 제안.
        - 시장 규모 분석에 따른 연간/분기별 판매 목표 설정.

---

- ssf ⇒ 리더님 진행
- 지그재그 → 이준님
- 29cm → 주아
- 에이블리 → 태중님
- 퀸잇 → 정태님

5개 플랫폼 상위 30개 여부에 따른 브랜드 스코어링 ⇒ **도씨 브랜드 분석 계획**

| Rank | Brand              | Total_Score | Appearances | Weighted_Score |
|------|--------------------|-------------|-------------|----------------|
| 1    | 도씨               | 274         | 3           | 822            |
| 2    | 쥬니쥬             | 199         | 1           | 199            |
| 3    | 오끼드             | 126         | 1           | 126            |
| 4    | 오프아워           | 123         | 1           | 123            |
| 5    | 레노마 언더웨어    | 117         | 1           | 117            |
| 6    | PALAN             | 103         | 1           | 103            |
| 7    | 뎁                 | 96          | 1           | 96             |
| 8    | 에이세             | 81          | 1           | 81             |
| 9    | 코지마켓           | 69          | 1           | 69             |
| 10   | 레노마 언더웨어    | 67          | 1           | 67             |


Raw Data

https://docs.google.com/spreadsheets/d/1E9dqXQ3hNUwIO3emvBv4dXhd3OuIh6GaF3HAlCIxVjA/edit?gid=827244988#gid=827244988


 .


<aside>
➡️ Total_Score = Total_Score = Σ(31 - Rank)
        각 브랜드가 플랫폼 내에서 획득한 순위 점수의 총합입니다. 높은 순위일수록 점수가 높습니다

➡️ Appearances = 해당 브랜드가 등장한 플랫폼의 수

➡️ Weighted_Score = Total_Score × Appearances
</aside>


---
2. 상품목록 및 리뷰 데이터 크롤링 코드 보완 및 완성

> 아래 코드는 퀸잇 (Seleniu 활용하여서 로컬 파이썬 환경에서만 진행가능)

In [ ]:
## SELENIUM을 활용한 상품데이터 크롤링 코드
#pip install selenium
#pip install selenium webdriver-manager

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

# 크롬 드라이버 자동 설정
service = Service(ChromeDriverManager().install())

# 크롬 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--new-window")  # 새로운 창 열기

# 웹드라이버 초기화
driver = webdriver.Chrome(service=service, options=options)

try:
    # 크롬 브라우저로 특정 URL 열기
    url = "https://web.queenit.kr/search?keyword=%ED%8C%8C%EC%9E%90%EB%A7%88"
    driver.get(url)

    # 페이지 로딩 대기
    time.sleep(3)

    # 스크롤 설정
    scroll_count = 10  # 스크롤 횟수 설정 (사용자가 변경 가능)
    scroll_unit = 5000  # 스크롤 단위

    for _ in range(scroll_count):
        driver.execute_script(f"window.scrollBy(0, {scroll_unit});")
        time.sleep(2)  # 스크롤 후 대기 시간

    # 데이터 저장용 리스트 초기화
    data = {
        "브랜드명": [],
        "상품명": [],
        "정가": [],
        "할인가": [],
        "할인율": [],
        "평점": [],
        "리뷰수": []
    }

    # 상품 정보 컨테이너 반복
    containers = driver.find_elements(By.CSS_SELECTOR, "div.css-j7qwjs")
    for container in containers:
        products = container.find_elements(By.CSS_SELECTOR, "a.css-8j52bx")
        for product in products:
            try:
                # 브랜드명
                try:
                    brand = product.find_element(By.CSS_SELECTOR, "div[variant='SpecificProductItemBrand']").text
                except:
                    brand = ""

                # 상품명
                try:
                    name = product.find_element(By.CSS_SELECTOR, "div[variant='SpecificProductItemProductName']").text
                except:
                    name = ""

                # 정가
                try:
                    price_original = product.find_element(By.CSS_SELECTOR, "div[variant='SubLabelXSmall']").text
                except:
                    price_original = ""

                # 할인가
                try:
                    price_discounted = product.find_element(By.CSS_SELECTOR, "div[variant='LabelSmall']").text
                except:
                    price_discounted = ""

                # 할인율
                try:
                    discount_rate = product.find_element(By.CSS_SELECTOR, "div[variant='SubLabelSmall']").text
                except:
                    discount_rate = ""

                # 평점 및 리뷰수
                try:
                    review_elements = product.find_elements(By.CSS_SELECTOR, "div[variant='SpecificProductItemReview']")
                    if len(review_elements) >= 2:
                        rating = review_elements[0].text.split('(')[0].strip()
                        reviews = review_elements[1].text.split('(')[-1].replace(")", "").strip()
                    else:
                        rating = ""
                        reviews = ""
                except:
                    rating = ""
                    reviews = ""

                # 데이터 추가
                data["브랜드명"].append(brand)
                data["상품명"].append(name)
                data["정가"].append(price_original)
                data["할인가"].append(price_discounted)
                data["할인율"].append(discount_rate)
                data["평점"].append(rating)
                data["리뷰수"].append(reviews)

            except Exception as e:
                print("Error processing product:", e)

    # 데이터프레임 생성
    df = pd.DataFrame(data)

    # 결과 출력
    print(df)

    # 데이터프레임을 엑셀 파일로 저장
    output_path = r"D:\\python workspace\\csv files\\product_data.xlsx"
    df.to_excel(output_path, index=False)
    print(f"Data saved to {output_path}")

finally:
    # 드라이버 종료
    driver.quit()


In [ ]:
### 리뷰데이터 크롤링 코드
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

# 크롬 드라이버 자동 설정
service = Service(ChromeDriverManager().install())

# 크롬 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--new-window")  # 새로운 창 열기

# 웹드라이버 초기화
driver = webdriver.Chrome(service=service, options=options)

try:
    # 크롬 브라우저로 특정 URL 열기
    url = "https://web.queenit.kr/product/ffcc433052f298cabca005da2d5c6bef"
    driver.get(url)

    # 페이지 로딩 대기
    time.sleep(3)

    # 특정 요소가 화면에 나타날 때까지 스크롤
    for _ in range(5):  # 최대 5번 스크롤
        try:
            clickable_element = driver.find_element(By.CSS_SELECTOR, "a.css-8pj6ef span.MuiTypography-root.MuiTypography-SpecificTabTextDefault.css-1mseikm")
            driver.execute_script("arguments[0].scrollIntoView(true);", clickable_element)
            time.sleep(0.5)  # 요소가 위치한 후 대기
            actions = ActionChains(driver)
            actions.move_to_element(clickable_element).click().perform()
            print("Element clicked successfully.")
            break
        except Exception:
            driver.execute_script("window.scrollBy(0, 1000);")
            time.sleep(2)

    # 스크롤 설정 (다시 스크롤 반복)
    scroll_unit = 5000
    for _ in range(10):
        driver.execute_script(f"window.scrollBy(0, {scroll_unit});")
        time.sleep(2)

    # 데이터 저장용 리스트 초기화
    data = {
        "평점": [],
        "리뷰작성일": [],
        "구매옵션": [],
        "구매평": [],
        "컬러평": [],
        "사이즈평": [],
        "두께평": []
    }

    # 리뷰 컨테이너 반복
    review_containers = driver.find_elements(By.CSS_SELECTOR, "div.css-j7qwjs[style*='background-color']")
    for review in review_containers:
        try:
            # 평점
            try:
                rating_container = review.find_element(By.CSS_SELECTOR, "div.review-rating-container.css-u4p24i")
                star_elements = rating_container.find_elements(By.CSS_SELECTOR, "img[src*='ic_star_full_16']")
                rating = len(star_elements)
            except:
                rating = ""

            # 리뷰작성일
            try:
                review_date = review.find_element(By.CSS_SELECTOR, "div.MuiBox-root.css-1qqq30y[variant='SubLabelXSmall']").text
            except:
                review_date = ""

            # 구매옵션
            try:
                purchase_option = review.find_element(By.CSS_SELECTOR, "div.MuiBox-root.css-1h7f5h8[variant='SubLabelSmall']").text
            except:
                purchase_option = ""

            # 구매평
            try:
                purchase_review = review.find_element(By.CSS_SELECTOR, "div.MuiBox-root.css-dcpr20[variant='BodyMedium']").text
            except:
                purchase_review = ""

            # 컬러평, 사이즈평, 두께평
            try:
                detail_container = review.find_element(By.CSS_SELECTOR, "div.css-13otrl7")
                color_review = detail_container.find_elements(By.CSS_SELECTOR, "div.MuiBox-root.css-ho64uk[variant='SubLabelSmall']")[0].text
                size_review = detail_container.find_elements(By.CSS_SELECTOR, "div.MuiBox-root.css-ho64uk[variant='SubLabelSmall']")[1].text
                thickness_review = detail_container.find_elements(By.CSS_SELECTOR, "div.MuiBox-root.css-ho64uk[variant='SubLabelSmall']")[2].text
            except:
                color_review, size_review, thickness_review = "", "", ""

            # 데이터 추가
            data["평점"].append(rating)
            data["리뷰작성일"].append(review_date)
            data["구매옵션"].append(purchase_option)
            data["구매평"].append(purchase_review)
            data["컬러평"].append(color_review)
            data["사이즈평"].append(size_review)
            data["두께평"].append(thickness_review)

        except Exception as e:
            print("Error processing review:", e)

    # 데이터프레임 생성
    df = pd.DataFrame(data)

    # 결과 출력
    print(df)

    # 데이터프레임을 엑셀 파일로 저장
    output_path = r"D:\\python workspace\\csv files\\review_data.xlsx"
    df.to_excel(output_path, index=False)
    print(f"Data saved to {output_path}")

finally:
    # 드라이버 종료
    driver.quit()


---
3. 데이터 분석방향 설정


## 텍스트 데이터 분석

<aside>
💡 텍스트 데이터 분석 프로세스

1. 데이터수집
    - 소셜 미디어, 리뷰, 뉴스, 대화 데이터 등 다양한 출처에서 텍스트 데이터를 수집.
2. 전처리
    - 불필요한 단어 제거(불용어 처리), 토큰화, 형태소 분석 등.
3. 분석 수행
    - 분석 목적에 따라 빈도 분석, 감정 분석, 네트워크 분석 등을 수행.
4. 시각화
    - 워드 클라우드, 네트워크 그래프, 감정 스코어 분포 등을 통해 결과를 효과적으로 전달.
</aside>

### 1. 빈도분석

- **개념**
    - 텍스트 데이터 내 특정 단어, 구문 또는 개체가 등장하는 빈도를 계산하여 주요 키워드 및 패턴을 분석하는 방법입니다.
- **활용 예시**
    - 고객 리뷰 데이터에서 가장 많이 언급되는 제품의 장점/단점을 파악.
    - 뉴스 기사에서 특정 이슈(예: "기후 변화", "AI 기술")의 언급 빈도를 분석하여 트렌드 확인.
- **주요 분석 지표**
    - 단어 빈도수 (Word Count)
    - TF-IDF (Term Frequency-Inverse Document Frequency)

### 2. 감정분석

- **개념**
    - 텍스트 데이터의 정서적 경향(긍정, 부정, 중립)을 파악하는 분석 기법입니다. 주로 자연어 처리(NLP) 기법과 사전 기반 또는 기계 학습 모델을 활용합니다.
- **활용 예시**
    - 소셜 미디어 게시물에서 브랜드에 대한 고객 감정 분석.
    - 영화 리뷰나 상품 리뷰에서 긍정/부정 평가의 비율 계산.
- **주요 분석 기법**
    - 감정 단어 사전 기반 분석 (e.g., NRC, VADER)
    - 딥러닝 모델 기반 감정 분류 (e.g., BERT, GPT 기반 모델)

### 3. ⭐️ 네트워크 분석 ⭐️

- **개념**
    - 텍스트 데이터에서 단어, 문장, 문서 간의 관계를 네트워크 형태로 모델링하여 분석하는 기법입니다.
        - 노드(Node): 단어, 구문, 개체 등
        - 엣지(Edge): 노드 간 연결 또는 연관성
- **분석 유형**
    - **연결강도**단어 또는 노드 간의 관계의 강도를 측정. 두 단어가 함께 나타나는 빈도수로 측정하거나, 상관 계수 기반으로 분석.
        - **활용 예시**
            - 고객 리뷰에서 "배터리"와 "지속시간"의 높은 연결강도를 확인하여 제품 개선에 활용.
            - 뉴스 데이터에서 특정 키워드 간의 연관성을 분석해 트렌드 도출.
    - **클러스터링**네트워크에서 서로 밀접히 연결된 노드 그룹을 파악.
        - **활용 예시**
            - 온라인 커뮤니티의 주제별 논의 클러스터 분류.
            - 논문 키워드 네트워크에서 연구 주제 도출

### 4. 노드 분석

- **개념**
    - 네트워크에서 개별 노드의 특성을 분석하여 중요도, 중심성 등을 파악하는 기법입니다.
- **주요 분석 지표**
    - **중심성(Centrality)**네트워크 내에서 특정 노드의 중요성을 나타냄.
        - **Degree Centrality**: 노드와 직접 연결된 엣지의 개수.
        - **Betweenness Centrality**: 노드가 다른 노드 간 최단 경로에 포함되는 정도.
        - **Closeness Centrality**: 노드가 다른 모든 노드와 얼마나 가까운가를 측정.
    - **PageRank**구글 검색 알고리즘의 핵심으로, 연결 구조에서 노드의 중요도를 평가.
- **활용 예시**
    - 논문 네트워크에서 가장 많이 인용된 논문 파악.
    - 소셜 미디어 데이터에서 인플루언서 또는 중심 키워드 도출